In [54]:
from __future__ import print_function
from pythtb import *
import numpy as np
import matplotlib.pyplot as plt
from itertools import product as itp

In [29]:
# define the lattice vectors and the coordinates of orbitals
lat = [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
orb = [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]

In [30]:
# define your model by making a 2D tight-binding checkboard model
my_model = tb_model(3,3,lat, orb)

In [31]:
# set model parameters
delta = 0.0
t = -0.6

In [32]:
# set on-site energies

# the first parameter includes a list of on-site energies for each orbital
# if this function is never called, the on-site energy is assumed to be zero
my_model.set_onsite([delta, delta, delta])

In [33]:
# set hoppings (one for each connected pair of orbitals)
# (hopping amplitude, i (index of orbital in the home unit cell), j, [lattice vector to cell containing j])

# hopping from the "first" orbital in the home unit cell to the "zeroth" orbital which is still in the home unit
# cell (indicated by the lattice vector [0, 0])
my_model.set_hop(t, 0, 0, [0, 1, 0])
my_model.set_hop(t, 0, 0, [1, 0, 0])

my_model.set_hop(t, 1, 1, [0, 1, 0])
my_model.set_hop(t, 1, 1, [0, 0, 1])

my_model.set_hop(t, 2, 2, [1, 0, 0])
my_model.set_hop(t, 2, 2, [0, 0, 1])

In [34]:
# print or display the tight-binding model
my_model.display()

---------------------------------------
report of tight-binding model
---------------------------------------
k-space dimension           = 3
r-space dimension           = 3
number of spin components   = 1
periodic directions         = [0, 1, 2]
number of orbitals          = 3
number of electronic states = 3
lattice vectors:
 #  0  ===>  [     1.0 ,     0.0 ,     0.0 ]
 #  1  ===>  [     0.0 ,     1.0 ,     0.0 ]
 #  2  ===>  [     0.0 ,     0.0 ,     1.0 ]
positions of orbitals:
 #  0  ===>  [     0.0 ,     0.0 ,     0.0 ]
 #  1  ===>  [     0.0 ,     0.0 ,     0.0 ]
 #  2  ===>  [     0.0 ,     0.0 ,     0.0 ]
site energies:
 #  0  ===>       0.0
 #  1  ===>       0.0
 #  2  ===>       0.0
hoppings:
<  0 | H |  0 + [  0 ,  1 ,  0 ] >     ===>     -0.6 +     0.0 i
<  0 | H |  0 + [  1 ,  0 ,  0 ] >     ===>     -0.6 +     0.0 i
<  1 | H |  1 + [  0 ,  1 ,  0 ] >     ===>     -0.6 +     0.0 i
<  1 | H |  1 + [  0 ,  0 ,  1 ] >     ===>     -0.6 +     0.0 i
<  2 | H |  2 + [  1 ,  0 ,  

In [8]:
# import triqs.lattice.utils
# from triqs.lattice.tight_binding import TBLattice
from triqs.lattice.utils import TB_from_pythTB
tb_triqs = TB_from_pythTB(my_model)
print(tb_triqs.units)

hoppingKeys = list(tb_triqs.hoppings.keys())
# np.array(hoppingKeys)

# for key in tb_triqs.hoppings.keys():
#     print(key)
#     print(tb_triqs.hoppings[key].real)
#     print()

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Starting serial run at: 2023-06-30 10:24:49.322768


In [55]:
from triqs.lattice.tight_binding import TBLattice
import sympy as sp


def sympyfy(TBLatObj):
    r"""
    returns the analytical form of the momentum space hamiltonian of the tight-binding model 
    from a tight-binding lattice object, by utilizing Fourier series
    
    Parameters
    ----------
    TBLatObj: triqs TBLattice object
        triqs tight binding object
        
    Returns
    -------
    Hk: NumPy Array
        the hamiltonian of the tight-binding model in momentum space
        
    """
    I = sp.I
    
    # axis directions in momentum space (dimension in momentum space is 3)
    kSpaceX, kSpaceY, kSpaceZ = sp.symbols("kx ky kz")
    
    # lattice matrix builder
    L1_1, L1_2, L1_3, L2_1, L2_2, L2_3, L3_1, L3_2, L3_3 = sp.symbols("L1_1 L1_2 L1_3 L2_1 L2_2 L2_3 L3_1 L3_2 L3_3")
    
    # converting the momentum space axial directions and elements of the lattice space matrix into matrices
    kSpaceMatrix = sp.Matrix([kSpaceX, kSpaceY, kSpaceZ])
    
    lattice = sp.Matrix([[L1_1, L1_2, L1_3],
                         [L2_1, L2_2, L2_3],
                         [L3_1, L3_2, L3_3]])

    # the number of orbitals
    numOrb = tb_triqs.n_orbitals
    
    TBLatObjHop = TBLatObj.hoppings
    
    # fourier transform the hoppings
    
    # the maximum x, y, and z directions represent the cutoff point we're going to look at
    # we ignore anything beyond that
    maxX, maxY, maxZ = list(np.max(np.array(list(TBLatObjHop.keys())), axis = 0))
    
    numCellsX, numCellsY, numCellsZ = [2*maxCoord + 1 for maxCoord in [maxX, maxY, maxZ]]
    
    # create an empty 5-dimensional matrix/tensor Hamiltonian, the real-space Hamiltonian that encapsulates 
    # the hoppings from one lattice site to another in real space
    # we use this to build up H(R) or H_R, one of the 2 terms used to determine the
    # Hamiltonian of the tight-binding model in momentum-space
    Hrij = np.zeros((numCellsX, numCellsY, numCellsZ, numOrb, numOrb), dtype = sp.exp)
    
    for key, hopping in TBLatObjHop.items():
        r0, r1, r2 = key
        Hrij[r0+maxX, r1+maxY, r2+maxZ] = hopping
    
    # constructing the second of the 2 terms needed to determine the Hamiltonian in momentum-space
    # this Hamiltonian is a summation over all real space axes and has the same dimensions as Hrij
    Hexp = np.empty_like(Hrij, dtype = sp.exp)
    
    # for xi in range(numCellsX):
    #     for yi in range(numCellsY):
    #         for zi in range(numCellsZ):
    #             r = np.array([xi - maxX, yi - maxY, zi - maxZ])
    #             r = lattice.dot(r)
    #             eikr = sp.exp(-I*kSpaceMatrix.dot(r))
    #             Hexp[xi, yi, zi, :, :] = eikr
    
    for xi, yi, zi in itp(range(numCellsX), range(numCellsY), range(numCellsZ)):
        r = np.array([xi - maxX, yi - maxY, zi - maxZ])
        r = lattice.dot(r)
        eikr = sp.exp(-I*kSpaceMatrix.dot(r))
        Hexp[xi, yi, zi, :, :] = eikr
    
    
    Hkrij = Hrij * Hexp
    Hk = np.sum(Hkrij, axis = (0, 1, 2))
    
    
    for i, j in itp(range(numOrb), repeat = 2):
        Hk[i, j] = Hk[i, j].rewrite(sp.cos).simplify()
    
    # Hk = Hk.rewrite(sp.cos).simplify()
    Hk = sp.Matrix(Hk)
    return Hk


In [56]:
print(sympyfy(tb_triqs))

Matrix([[-1.2*cos(L1_1*kx + L2_1*ky + L3_1*kz) - 1.2*cos(L1_2*kx + L2_2*ky + L3_2*kz), 0, 0], [0, -1.2*cos(L1_2*kx + L2_2*ky + L3_2*kz) - 1.2*cos(L1_3*kx + L2_3*ky + L3_3*kz), 0], [0, 0, -1.2*cos(L1_1*kx + L2_1*ky + L3_1*kz) - 1.2*cos(L1_3*kx + L2_3*ky + L3_3*kz)]])


In [58]:
print(tb_triqs.units)
print(tb_triqs.
print(type(tb_triqs))

<class 'triqs.lattice.tight_binding.TBLattice'>


In [44]:
# axis directions in momentum space (dimension in momentum space is 3)
kx, ky, kz = sp.symbols("kx ky kz")

# lattice matrix builder
L1_1, L1_2, L1_3, L2_1, L2_2, L2_3, L3_1, L3_2, L3_3 = sp.symbols("L1_1 L1_2 L1_3 L2_1 L2_2 L2_3 L3_1 L3_2 L3_3")

# converting the momentum space axial directions and elements of the lattice space matrix into matrices
kSpaceMatrix = sp.Matrix([kx, ky, kz])

lattice = sp.Matrix([[L1_1, L1_2, L1_3],
                     [L2_1, L2_2, L2_3],
                     [L3_1, L3_2, L3_3]])

Hk = sympyfy(tb_triqs)
ak, bk, ck = sp.symbols("ak bk ck")
# expr.subs(x, y)
Hk = Hk.subs(L1_1*kx + L2_1*ky + L3_1*kz, ak)
Hk = Hk.subs(L1_2*kx + L2_2*ky + L3_2*kz, bk)
Hk = Hk.subs(L1_3*kx + L2_3*ky + L3_3*kz, ck)
print(Hk)


Matrix([[-1.2*cos(ak) - 1.2*cos(bk), 0, 0], [0, -1.2*cos(bk) - 1.2*cos(ck), 0], [0, 0, -1.2*cos(ak) - 1.2*cos(ck)]])


In [ ]:
# generate the k-point path and labels
path= [[0.0,0.0,0.0],[0.5,0.0, 0.0],[0.5,0.5,0.0],[0.0,0.0,0.0]]
label = (r'$\Gamma $',r'$X$', r'$M$', r'$\Gamma $')
(k_vec, k_dist, k_node) = my_model.k_path(path, 101, report=False)


In [ ]:
# import triqs.lattice.utils
# from triqs.lattice.tight_binding import TBLattice
from triqs.lattice.utils import TB_from_pythTB
tb_triqs = TB_from_pythTB(my_model)
# print(tb_triqs)


for key in tb_triqs.hoppings.keys():
    print(key)
    print(tb_triqs.hoppings[key].real)
    print()

In [ ]:
# solve for the eigenenergies of the Hamiltonian on the set of k-points above
evals = my_model.solve_all(k_vec)
print(evals)

In [ ]:
# plotting of band structure
print('Plotting the band structure...')
fig, ax = plt.subplots()
ax.set_xlim(k_node[0], k_node[-1])
ax.set_xticks(k_node)
ax.set_xticklabels(label)
print(k_node)

for n in range(len(k_node)):
    ax.axvline(x=k_node[n], linewidth=0.5, color='k')

# plot the bands
for n in range(2):
    ax.plot(k_dist, evals[n])

# put title
ax.set_title('Checkerboard band structure')
ax.set_xlabel('Path in k-space')
ax.set_ylabel('Band energy')

fig.savefig('2D Tight-Binding Checkerboard Model.pdf')